In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/time-series-tools/pycatch22-0.4.2.tar.gz
/kaggle/input/time-series-tools/multiprocess-0.70.14-py37-none-any.whl
/kaggle/input/time-series-tools/tsflex-0.3.0-py3-none-any.whl
/kaggle/input/time-series-tools/joblib-1.2.0-py3-none-any.whl
/kaggle/input/time-series-tools/scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/time-series-tools/six-1.16.0-py2.py3-none-any.whl
/kaggle/input/time-series-tools/__results__.html
/kaggle/input/time-series-tools/threadpoolctl-3.1.0-py3-none-any.whl
/kaggle/input/time-series-tools/python_dateutil-2.8.2-py2.py3-none-any.whl
/kaggle/input/time-series-tools/pytz-2022.7.1-py2.py3-none-any.whl
/kaggle/input/time-series-tools/tqdm-4.65.0-py3-none-any.whl
/kaggle/input/time-series-tools/dill-0.3.6-py3-none-any.whl
/kaggle/input/time-series-tools/__notebook__.ipynb
/kaggle/input/time-series-tools/scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
/kaggle/input/time-series-tools/pandas-1.3.

## Importing Libraries

In [3]:
!pip install tsflex --no-index --find-links=file:///kaggle/input/time-series-tools
!pip install seglearn --no-index --find-links=file:///kaggle/input/time-series-tools

Looking in links: file:///kaggle/input/time-series-tools
Processing /kaggle/input/time-series-tools/tsflex-0.3.0-py3-none-any.whl
Looking in links: file:///kaggle/input/time-series-tools
Processing /kaggle/input/time-series-tools/seglearn-1.2.5-py3-none-any.whl


## Importing Libraries

In [4]:
#General Libraries

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn import *
import lightgbm as lgb
import glob
import pathlib

#ts Libraries
from seglearn.feature_functions import base_features, emg_features
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper

# Sklearn libraries
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import average_precision_score
from sklearn.base import clone



## Defining Paths

In [5]:
path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

In [6]:
TraingData = glob.glob(path+'train/**/**')
test = glob.glob(path+'test/**/**')
subjects = pd.read_csv(path+'subjects.csv')
tdcsfog_metadata=pd.read_csv(path+'tdcsfog_metadata.csv')
defog_metadata=pd.read_csv(path+'defog_metadata.csv')
tasks = pd.read_csv(path+'tasks.csv')
sampleSub = pd.read_csv(path+'sample_submission.csv')


## Creating tdcsfog and defog  cols

In [7]:
tdcsfog_metadata['Session']='tdcsfog'
defog_metadata['Session']='defog'
FullMeta=pd.concat([tdcsfog_metadata,defog_metadata])

## Adding Task Duriation col

In [8]:

tasks['Complete'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Complete'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[-1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['Time_K'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(tasks[tasks.columns[1:]])


In [9]:
tasks

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST,Time_K
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320,2
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698,0
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299,3
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351,7
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960,3
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889,3
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152,3
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831,7


## Creating Subject Groups

In [10]:

subjects = subjects.fillna(0).groupby('Subject').median()
subjects = subjects.reset_index()
subjects['Subject_K'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(subjects[subjects.columns[1:]])
subjects=subjects.rename(columns={'Visit':'s_Visit','Age':'s_Age','YearsSinceDx':'s_YearsSinceDx','UPDRSIII_On':'s_UPDRSIII_On','UPDRSIII_Off':'s_UPDRSIII_Off','NFOGQ':'s_NFOGQ'})


In [11]:
subjects

,Subject,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,Subject_K
0,04fcdb,1.0,63.0,3.0,30.0,0.0,0.0,3
1,05595e,1.0,56.0,8.0,28.0,0.0,0.0,3
2,0967b2,1.5,59.0,10.0,37.5,46.0,16.0,6
3,097078,0.0,70.0,10.0,27.0,50.0,20.0,0
4,0e0908,0.0,67.0,6.0,16.0,61.0,21.0,0
...,...,...,...,...,...,...,...,...
131,f80d3a,1.5,78.0,18.0,23.5,29.5,17.5,0
132,f90887,1.5,72.0,16.0,29.0,43.5,24.0,6
133,fc1e1b,1.0,82.0,11.0,38.0,42.0,21.0,2
134,fe5d84,1.5,72.0,14.0,22.5,39.0,16.0,0


## Adding Rest of Features

In [12]:
FullFeattureList=['Visit','Test','Medication','s_Visit','s_Age','s_YearsSinceDx','s_UPDRSIII_On','s_UPDRSIII_Off','s_NFOGQ','Subject_K']
MetaFeattureList=FullMeta.merge(subjects,how='left',on='Subject').copy()
MetaFeattureList['Medication']=MetaFeattureList['Medication'].factorize()[0]

In [13]:
MetaFeattureList

,Id,Subject,Visit,Test,Medication,Session,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,Subject_K
0,003f117e14,13abfd,3,2.0,0,tdcsfog,0.0,68.0,9.0,17.0,15.0,15.0,8
1,009ee11563,d81e3a,4,2.0,0,tdcsfog,0.0,71.0,10.0,42.0,0.0,24.0,1
2,011322847a,203e85,2,2.0,0,tdcsfog,0.0,67.0,12.0,27.0,28.0,19.0,0
3,01d0fe7266,203e85,2,1.0,1,tdcsfog,0.0,67.0,12.0,27.0,28.0,19.0,0
4,024418ba39,cecfb8,19,3.0,0,tdcsfog,0.0,60.0,7.0,30.0,0.0,19.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,ce8b0b,1,NaN,1,defog,1.5,65.0,7.0,50.0,59.5,24.5,2
966,f40e8c6ebe,d9529b,1,NaN,1,defog,1.0,28.0,4.0,54.0,50.0,25.0,9
967,f8ddbdd98d,fc1e1b,1,NaN,0,defog,1.0,82.0,11.0,38.0,42.0,21.0,2
968,f9efef91fb,fe5d84,2,NaN,1,defog,1.5,72.0,14.0,22.5,39.0,16.0,0


In [14]:
best_params_ = {'estimator__colsample_bytree': 0.6, 'estimator__learning_rate': 0.1, 'estimator__max_depth': 10, 
 'estimator__min_child_weight': 2, 'estimator__n_estimators': 32,  'estimator__subsample': 0.95}

In [15]:
best_params_ = {pp: v for p, v in best_params_.items() for pp in p.split('__')}; del best_params_['estimator']



## Creating Time Series Features

In [16]:
RegularFeatures = MultipleFeatureDescriptors(windows=[5_000],strides=[5_000],functions=seglearn_feature_dict_wrapper(base_features()),series_names=['AccV', 'AccML', 'AccAP'],
)


In [17]:
EMG = emg_features()
del EMG['simple square integral']
EMG = MultipleFeatureDescriptors( functions=seglearn_feature_dict_wrapper(EMG),windows=[5_000],strides=[5_000],series_names=['AccV', 'AccML', 'AccAP'],
)

In [18]:
FullCollectionFeatures= FeatureCollection([RegularFeatures, EMG])

## Taking Features with Time Feature


In [19]:
def dataMaker(file):
    try:
        dataFrame = pd.read_csv(file, index_col="Time", usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])
        dataFrame['Id'] = file.split('/')[-1].split('.')[0]
        dataFrame['Session'] = pathlib.Path(file).parts[-2]        
        dataFrame['Fractions']=(dataFrame.index/dataFrame.index.max()).values
        dataFrame = pd.merge(dataFrame, tasks[['Id','Time_K']], how='left', on='Id').fillna(-1)
        dataFrame = pd.merge(dataFrame, MetaFeattureList[['Id','Subject']+['Visit','Test','Medication','Subject_K']], how='left', on='Id').fillna(-1)
        dataFrameFeatures = FullCollectionFeatures.calculate(dataFrame, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        dataFrame = dataFrame.merge(dataFrameFeatures, how="left", left_index=True, right_index=True)
        dataFrame.fillna(method="ffill", inplace=True)
        return dataFrame
    except: pass


## Gathering Training Data

In [20]:
TraingData = pd.concat([dataMaker(file) for file in tqdm(TraingData)]).fillna(0)


  0%|          | 0/970 [00:00<?, ?it/s]

In [21]:
TraingData=TraingData.reset_index(drop=True)

In [22]:
TraingData.head(10)

,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Session,Fractions,Time_K,...,AccV__minimum__w=5000,AccV__mse__w=5000,AccV__root_mean_square__w=5000,AccV__skew__w=5000,AccV__slope_sign_changes__w=5000,AccV__std__w=5000,AccV__var__w=5000,AccV__waveform_length__w=5000,AccV__willison_amplitude__w=5000,AccV__zero_crossing__w=5000
0,-1.002697,0.022371,0.068304,0,0,0,be9d33541d,defog,0.000000,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
1,-1.002641,0.019173,0.066162,0,0,0,be9d33541d,defog,0.000008,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
2,-0.999820,0.019142,0.067536,0,0,0,be9d33541d,defog,0.000017,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
3,-0.998023,0.018378,0.068409,0,0,0,be9d33541d,defog,0.000025,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
4,-0.998359,0.016726,0.066448,0,0,0,be9d33541d,defog,0.000034,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
5,-1.002969,0.016203,0.065118,0,0,0,be9d33541d,defog,0.000042,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
6,-1.010631,0.014523,0.062518,0,0,0,be9d33541d,defog,0.000050,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
7,-1.015932,0.014735,0.056944,0,0,0,be9d33541d,defog,0.000059,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
8,-1.016709,0.020147,0.054147,0,0,0,be9d33541d,defog,0.000067,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0
9,-1.016231,0.022617,0.055152,0,0,0,be9d33541d,defog,0.000076,0.0,...,-2.295443,4976.811035,0.997678,-3.076039,1459.0,0.10962,0.012017,75.765259,4999.0,4999.0


In [23]:
TraingData.columns

Index(['AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn', 'Walking', 'Id',
       'Session', 'Fractions', 'Time_K', 'Subject', 'Visit', 'Test',
       'Medication', 'Subject_K', 'AccAP__abs_energy__w=5000',
       'AccAP__abs_sum__w=5000', 'AccAP__emg_var__w=5000',
       'AccAP__kurt__w=5000', 'AccAP__maximum__w=5000', 'AccAP__mean__w=5000',
       'AccAP__mean_abs__w=5000', 'AccAP__mean_crossings__w=5000',
       'AccAP__median__w=5000', 'AccAP__minimum__w=5000', 'AccAP__mse__w=5000',
       'AccAP__root_mean_square__w=5000', 'AccAP__skew__w=5000',
       'AccAP__slope_sign_changes__w=5000', 'AccAP__std__w=5000',
       'AccAP__var__w=5000', 'AccAP__waveform_length__w=5000',
       'AccAP__willison_amplitude__w=5000', 'AccAP__zero_crossing__w=5000',
       'AccML__abs_energy__w=5000', 'AccML__abs_sum__w=5000',
       'AccML__emg_var__w=5000', 'AccML__kurt__w=5000',
       'AccML__maximum__w=5000', 'AccML__mean__w=5000',
       'AccML__mean_abs__w=5000', 'AccML__mean_crossings__w=50

In [24]:
TraingData.describe

<bound method NDFrame.describe of               AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
0        -1.002697  0.022371  0.068304                0     0        0   
1        -1.002641  0.019173  0.066162                0     0        0   
2        -0.999820  0.019142  0.067536                0     0        0   
3        -0.998023  0.018378  0.068409                0     0        0   
4        -0.998359  0.016726  0.066448                0     0        0   
...            ...       ...       ...              ...   ...      ...   
20588369 -9.915920 -0.105897 -1.123455                0     0        0   
20588370 -9.693752 -0.066892 -1.114903                0     0        0   
20588371 -9.548118 -0.098315 -1.112123                0     0        0   
20588372 -9.469803 -0.111004 -1.130814                0     0        0   
20588373 -9.566318 -0.113865 -1.103353                0     0        0   

                  Id  Session  Fractions  Time_K  ... AccV__minimum__w=5000  

In [25]:
responseCols = ['StartHesitation', 'Turn' , 'Walking']

In [26]:
len(responseCols)

3

## Adding Time Fraction

In [27]:
for i in range(len(responseCols)):
        TraingData.loc[TraingData[responseCols[i]]==1,'Fractions']

## Defining Prediction and Model 

In [28]:
def precisionCalculation(groundTruth,yPrime):
    Avgprecision = average_precision_score(groundTruth, yPrime)
    return 'average_precision', Avgprecision, True

In [29]:
class Model(MultiOutputRegressor):
    def fit(self, inputs, outputs, eval_set=None, **fit):
        self.estimators_ = [clone(self.estimator) for _ in range(outputs.shape[1])]
        for i, estimator in enumerate(self.estimators_):
            if eval_set:
                fit['eval_set'] = [(eval_set[0], eval_set[1][:, i])]
            estimator.fit(inputs, outputs[:, i], **fit)
        return self

## Model and Submission Params

In [30]:
numOfFold=5
folds = GroupKFold(numOfFold)
regressionModel=[]
crossVerifySets=[]
results=[]
sampleSub['t'] = 0
submission = []

## Grouping Variables by Subject aka patient

In [31]:
grouping = TraingData.Subject

In [32]:
Allgroups=folds.split(TraingData, groups=grouping)

In [33]:
SubmitCols = ['Id', 'StartHesitation', 'Turn' , 'Walking']
columns = [c for c in TraingData.columns if c not in ['Id','Subject','Session', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]


In [34]:
for fold, (trainID,testID ) in enumerate(tqdm(Allgroups, total=numOfFold, desc="Showing Folds Progress")):
    trainID=pd.Series(trainID).sample(n=2000000,random_state=20).values 
    start = lgb.LGBMRegressor(**best_params_)
    model = Model(start)
    TrainX,TrainY=TraingData.loc[trainID,columns].to_numpy(),TraingData.loc[trainID,responseCols].to_numpy()
    TestX,TestY=TraingData.loc[testID,columns].to_numpy(),TraingData.loc[testID,responseCols].to_numpy()
    model.fit(TrainX,TrainY,eval_set=(TestX,TestY),eval_metric=precisionCalculation)
    regressionModel.append(model)
    crossVerify=metrics.average_precision_score(TestY, model.predict(TestX).clip(0.0,1.0))
    crossVerifySets.append(crossVerify)
print(crossVerifySets)

Showing Folds Progress:   0%|          | 0/5 [00:00<?, ?it/s]

[1]	valid_0's l2: 0.0578713	valid_0's average_precision: 0.130274
[2]	valid_0's l2: 0.0578203	valid_0's average_precision: 0.239192
[3]	valid_0's l2: 0.0578027	valid_0's average_precision: 0.256744
[4]	valid_0's l2: 0.057818	valid_0's average_precision: 0.25399
[5]	valid_0's l2: 0.0577161	valid_0's average_precision: 0.310761
[6]	valid_0's l2: 0.0576393	valid_0's average_precision: 0.337021
[7]	valid_0's l2: 0.0576829	valid_0's average_precision: 0.315775
[8]	valid_0's l2: 0.0576275	valid_0's average_precision: 0.303809
[9]	valid_0's l2: 0.057518	valid_0's average_precision: 0.269983
[10]	valid_0's l2: 0.0575344	valid_0's average_precision: 0.259943
[11]	valid_0's l2: 0.0575016	valid_0's average_precision: 0.262878
[12]	valid_0's l2: 0.0575088	valid_0's average_precision: 0.261121
[13]	valid_0's l2: 0.0575215	valid_0's average_precision: 0.253464
[14]	valid_0's l2: 0.0574354	valid_0's average_precision: 0.265897
[15]	valid_0's l2: 0.0572217	valid_0's average_precision: 0.2779
[16]	vali

In [35]:

for file in test:
    dataFrame = pd.read_csv(file)
    dataFrame.set_index('Time', drop=True, inplace=True)

    dataFrame['Id'] = file.split('/')[-1].split('.')[0]
    dataFrame['Fractions']=(dataFrame.index/dataFrame.index.max()).values
    dataFrame = pd.merge(dataFrame, tasks[['Id','Time_K']], how='left', on='Id').fillna(-1)
    dataFrame = pd.merge(dataFrame, MetaFeattureList[['Id','Subject']+['Visit','Test','Medication','Subject_K']], how='left', on='Id').fillna(-1)
    dataFrameFeatures = FullCollectionFeatures.calculate(dataFrame, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin")
    dataFrame = dataFrame.merge(dataFrameFeatures, how="left", left_index=True, right_index=True)
    dataFrame.fillna(method="ffill", inplace=True)

    
    results=[]
    for i_fold in range(numOfFold):
        re=np.round(regressionModel[i_fold].predict(dataFrame[columns]).clip(0.0,1.0),3)
        results.append(np.expand_dims(re,axis=2))
    results=np.mean(np.concatenate(results,axis=2),axis=2)
    answer = pd.DataFrame(results, columns=responseCols)
    
    dataFrame = pd.concat([dataFrame,answer], axis=1)
    dataFrame['Id'] = dataFrame['Id'].astype(str) + '_' + dataFrame.index.astype(str)
    submission.append(dataFrame[SubmitCols])
    
    


## Converting Results To CSV

In [36]:
submission = pd.concat(submission)
submission = pd.merge(sampleSub[['Id']], submission, how='left', on='Id').fillna(0.0)
submission[SubmitCols].to_csv('submission.csv', index=False)

In [37]:
submission

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.0164,0.0170,0.0030
1,003f117e14_1,0.0164,0.0170,0.0030
2,003f117e14_2,0.0164,0.0170,0.0030
3,003f117e14_3,0.0164,0.0170,0.0030
4,003f117e14_4,0.0164,0.0170,0.0030
...,...,...,...,...
286365,02ab235146_281683,0.0008,0.0344,0.0074
286366,02ab235146_281684,0.0008,0.0344,0.0074
286367,02ab235146_281685,0.0008,0.0344,0.0074
286368,02ab235146_281686,0.0008,0.0344,0.0074
